# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Постройте модель с максимально большим значением *accuracy*.

## Откройте и изучите файл

In [1]:
import pandas as pd
import math as mt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

display(df.head(), df.info(), df.describe(), df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


None

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

### Вывод

- В данных нет пропусков.
- Значения в столбце 'minutes' округлены в большую сторону, по правилам оператора
- Замены типы данных в столбцах 'calls', 'messages', 'minutes'

## Разбейте данные на выборки

In [3]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

# Выделяем тестовую выбоку - 20% от всех данных
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345)

# выделяем валидационную выборку из обучающей выборки - 20% от всех данных, но уже из 80% данных
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train, target_train, test_size=0.25, random_state=12345)

- Исходные данные были поделены на обучающие - 60%, валидационные - 20% и тестовые - 20%

In [4]:
features_train.shape, features_valid.shape, features_test.shape

((1928, 4), (643, 4), (643, 4))

## Исследуйте модели

### Дерево решений

In [5]:
# обучение модели и проверка качества на валидационной выборке
best_model_tree = None
best_result = 0
best_depth = 0
for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions)
    if result > best_result:
        best_model_tree = model
        best_result = result
        best_depth = depth
print("Accuracy лучшей модели:", best_result)
print("Max_depth:", best_depth)

Accuracy лучшей модели: 0.7744945567651633
Max_depth: 7


### Случайный лес

In [6]:
best_model_forest = None
best_result = 0
best_est = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model_forest = model
        best_result = result
        best_est = est

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("n_estimators:", best_est)

Accuracy наилучшей модели на валидационной выборке: 0.7884914463452566
n_estimators: 10


### Логистическая регрессия

In [7]:
model_log_regres = LogisticRegression(random_state=12345)
model_log_regres.fit(features_train, target_train)
result = model_log_regres.score(features_valid, target_valid)

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.6967340590979783


### Вывод

- Была проведена проверка качества моделей на валидационной выборке.
- Для дерева решений оптимальные значения: max_depth = 5, accuracy = 0.774.
- Для случайного леса оптимальные значения: n_estimators = 10, accuracy = 0.788.
- Для логистической регрессии оптимальное значение: accuracy = 0.697.

<b> - Модель случайного леса показала лучшей результат качества и выше минимально необходимого: 0.75

## Проверьте модель на тестовой выборке

In [8]:
def test_model(model_name, test_data, target_data):
    
    """Функция выводит значение качества модели при проверке на тестовой выборке"""
    
    predictions = model_name.predict(test_data)
    result_test = accuracy_score(target_data, predictions)
    print("Accuracy модели:", result_test)

In [9]:
test_model(best_model_tree, features_test, target_test)

Accuracy модели: 0.7884914463452566


In [10]:
test_model(best_model_forest, features_test, target_test)

Accuracy модели: 0.7869362363919129


In [11]:
test_model(model_log_regres, features_test, target_test)

Accuracy модели: 0.702954898911353


- Качество модолеей дерева решений и случайного леса практически одинаковы на тестовой выборке и больше 0.75. Модель логистичекой регресси значительно отстаёт, почти на 9%.

## (бонус) Проверьте модели на адекватность

In [12]:
from sklearn.metrics import confusion_matrix

In [13]:
t_n = []
f_p = []
f_n = []
t_p = []

for model in [best_model_tree, best_model_forest, model_log_regres]:
    predictions = model.predict(features_test)
    tn, fp, fn, tp = confusion_matrix(target_test, predictions).ravel()
    t_n.append(tn), f_p.append(fp), f_n.append(fn), t_p.append(tp)
    
matrix_data = {'tn': t_n, 'fp': f_p, 'fn': f_n, 'tp': t_p}
pd.DataFrame(data=matrix_data, index=['best_model_tree', 'best_model_forest', 'model_log_regres'])

,tn,fp,fn,tp
best_model_tree,421,26,110,86
best_model_forest,406,41,96,100
model_log_regres,443,4,187,9


- TP: True Positive: прогнозируемые значения, правильно прогнозируемые как фактические положительные
- FP: Предсказанные значения неправильно предсказывают фактический положительный результат. т.е. отрицательные значения прогнозируются как положительные
- FN: False Negative: положительные значения прогнозируются как отрицательные
- TN: True Negative: прогнозируемые значения, правильно прогнозируемые как фактические негативные

### Вывод

- Общее качество моделей дерева решений и случайного леса почти одинаковое: 0.788.
- Если же анализировать матрицу путаницы, то для модели случайного леса ошибки в предсказании тарифа "Ультра" в 2 раза выше чем у дерева решения. Т.е. дать неправильную рекомендацию по тарифу Ультра модель случайного леса может дать для 6 человек из 100, а дерево решений только для 3. Тариф Ультра дороже и значит может принести больше выручки.

In [14]:
X = best_model_tree.predict(features_test) # предсказания нашей модели
y = target_test # фактическое значение целевого параметра

for i in range(0, 2):
    dummy_clf = DummyClassifier(strategy="constant", constant=i)
    dummy_clf.fit(X, y)
    print(np.unique(dummy_clf.predict(X), return_counts=True), dummy_clf.score(X, y))

(array([0]), array([643])) 0.6951788491446346
(array([1]), array([643])) 0.3048211508553655
